# OpenCV 视频抓取

1. 构建一个项目工程
    1. 创建目录/文件结构
    2. 工程组织文件（Makefile $\to$ QMake）
        - main.pro

2. 创建Qt应用

- 根据前两天所学，做好基本套路

3. UI

   - 利用ui工具设计简单的ui，并保存

4. 多线程

5. 视频采集
    - 使用信号发送给窗体显示
        1. 定义信号-> 发送：
        2. 接受信号-> 显示

-----

- OpenCV的视频采集的相关的类
    - 构造器
    - 成员函数
        - 使用规律：
            - 打开摄像头
            - 读取图像
            

6.代码展示

- cvvideo.h

```C++
#ifndef CV_VIDEO_H
#define CV_VIDEO_h
#include <QtCore/QThread>
#include <opencv2/opencv.hpp>

class CVVideo:public QThread{
    Q_OBJECT
public:
    CVVideo(QObject *parent=0);
    ~CVVideo();

protected:
    virtual void run();

private:
    cv::VideoCapture *dev;
    cv::CascadeClassifier *classifier;

signals:
    void sig_video(cv::Mat img);

};

#endif
```

- cvvideo.cpp

```C++
#include "cvvideo.h"
#include <iostream>

CVVideo::CVVideo(QObject *parent):
    QThread(parent),
    dev(new cv::VideoCapture(0, cv::CAP_DSHOW)){
    classifier = new cv::CascadeClassifier("haarcascade_frontalface_alt2.xml");

}

CVVideo::~CVVideo(){
    dev->release();
    delete classifier;
    delete dev;
}

void CVVideo::run(){
    while(true){
        //视频采集
        cv::Mat img;
        dev->read(img);

        // 图像如何处理
        //侦测人脸
        std::vector<cv::Rect> objs; // 用来存储识别的人脸
        std::vector<int> rejectLeves; // 返回拒绝的level
        std::vector<double> levelWeights; // level的权重

        classifier->detectMultiScale(
            img, //输入图像
            objs,
            rejectLeves,
            levelWeights,
            1.05, // 人脸搜索的模板
            3, //最小邻域个数限制
            1, //边缘检测（Canny）
            cv::Size(),
            cv::Size(),
            true
        );
        // 标记人脸
        if(!objs.empty()){
            for(int i = 0; i < objs.size(); i ++){
                if(levelWeights[i] >= 55.5){
                    cv::rectangle(img, objs[i], cv::Scalar(0, 0, 255), 2);
                }
            }
        }
        //视频图像发送到窗体显示
        emit sig_video(img);
        QThread::usleep(100000);
    }
}
```

- cvdialog.h

```C++
#ifndef CV_VIDEO_H
#define CV_VIDEO_h
#include <QtCore/QThread>
#include <opencv2/opencv.hpp>

class CVVideo:public QThread{
    Q_OBJECT
public:
    CVVideo(QObject *parent=0);
    ~CVVideo();

protected:
    virtual void run();

private:
    cv::VideoCapture *dev;
    cv::CascadeClassifier *classifier;

signals:
    void sig_video(cv::Mat img);

};

#endif

```

- cvdialog.cpp

```C++
#include "cvdialog.h"
#include <iostream>
CVDialog::CVDialog(QWidget *parent):
    QDialog(parent),
    ui(new Ui::video()),
    video(new CVVideo()),
    type(0){
        ui->setupUi(this);
        QObject::connect(video, SIGNAL(sig_video(cv::Mat)), this, SLOT(show_video(cv::Mat)));
        video->start();
}

CVDialog::~CVDialog(){
    video->terminate();
    delete ui;
    delete video;
}

void CVDialog::gray(){
    type = 1;
}

void CVDialog::gauss(){
    type = 2;
}

void CVDialog::show_video(cv::Mat img){
    //添加图像处理
    if(type == 0){
        // 显示图像
        // 1.QImage
        QImage i_out(img.data, img.cols, img.rows, QImage::Format_BGR888);
        // 2.QPixmap
        QPixmap p_out = QPixmap::fromImage(i_out);
        // 3.显示到标签框
        ui->lbl_video->setPixmap(p_out);
        // 4.根据标签框大小显示图像
        ui->lbl_video->setScaledContents(true);
    }
    if(type == 1){
        cv::cvtColor(img, img, cv::COLOR_BGR2GRAY);
        // 1.QImage
        QImage i_out(img.data, img.cols, img.rows, QImage::Format_Grayscale8);
        // 2.QPixmap
        QPixmap p_out = QPixmap::fromImage(i_out);
        // 3.显示到标签框
        ui->lbl_video->setPixmap(p_out);
        // 4.根据标签框大小显示图像
        ui->lbl_video->setScaledContents(true);
    }
    if(type == 2){
        // 高斯变换
        cv::GaussianBlur(img, img, cv::Size2i(17, 17), 3.0);
        // cv::Sobel(img, img, -1, 1, 1, 3,128); sobel变换
        // 1.QImage
        QImage i_out(img.data, img.cols, img.rows, QImage::Format_BGR888);
        // 2.QPixmap
        QPixmap p_out = QPixmap::fromImage(i_out);
        // 3.显示到标签框
        ui->lbl_video->setPixmap(p_out);
        // 4.根据标签框大小显示图像
        ui->lbl_video->setScaledContents(true);
    }
}
```

- main.pro

```C++
# 控制编译器c++相关的编译选项
QMAKE_CXXFLAGS += /source-charset:utf-8
QMAKE_CXXFLAGS += /execution-charset:utf-8

# 配置程序的类型：应用/共享库
TEMPLATE = app

# 基本配置
CONFIG += debug
CONFIG += console
CONFIG += thread
CONFIG += qt

#opencv的链接的相关的库与头文件
INCLUDEPATH +="E:\OPENCV\install\include"

LIBS += -L"E:\OPENCV\install\x64\vc16\lib"
LIBS += -lopencv_core420d
LIBS += -lopencv_videoio420d
LIBS += -lopencv_imgcodecs420d
LIBS += -lopencv_imgproc420d
LIBS += -lopencv_objdetect420d

# Qt的模块库的配置
QT += core
QT += gui
QT += widgets

# 工程代码文件
SOURCES += main.cpp
SOURCES += cvvideo.cpp
SOURCES += cvdialog.cpp

HEADERS += cvdialog.h
HEADERS += cvvideo.h

FORMS += video.ui

# 输出的执行文件
TARGET += main


```

# OpenCV图像特征处理

- $\texttt{dst} = \frac{\partial^{xorder+yorder} \texttt{src}}{\partial x^{xorder} \partial y^{yorder}}$

1. 灰度变换
```C++
     cv::cvtColor(img, img, cv::COLOR_BGR2GRAY);
     QImage i_out(img.data, img.cols, img.rows, QImage::Format_Grayscale8);
     // 2. QPixmap
     QPixmap p_out = QPixmap::fromImage(i_out); 
     // 3. 显示到标签框
     ui-&gt;lbl_video-&gt;setPixmap(p_out);
     // 4. 根据标签框大小显示图像内容
     ui-&gt;lbl_video-&gt;setScaledContents(true);
```
2. Sobel变换
```C++
        cv::Mat sobel_img; 
        cv::Sobel(img, sobel_img, -1, 1, 0, 3, 1, 128); 
        QImage i_out(sobel_img.data, sobel_img.cols, sobel_img.rows, QImage::Format_BGR888);
        // 2. QPixmap 
        QPixmap p_out = QPixmap::fromImage(i_out); 
        // 3. 显示到标签框 
        ui->lbl_video->setPixmap(p_out); 
        // 4. 根据标签框大小显示图像内容 
        ui->lbl_video->setScaledContents(true);
```
3. 高斯变换
```C++
        cv::GaussianBlur(img, img, cv::Size2i(17, 17), 3.0);
        // cv::Sobel(img, img, -1, 1, 1, 3,128); sobel变换
        // 1.QImage
        QImage i_out(img.data, img.cols, img.rows, QImage::Format_BGR888);
        // 2.QPixmap
        QPixmap p_out = QPixmap::fromImage(i_out);
        // 3.显示到标签框
        ui->lbl_video->setPixmap(p_out);
        // 4.根据标签框大小显示图像
        ui->lbl_video->setScaledContents(true);
```
4. Filter2D：卷积运算
```C++
     cv::Mat o_img;
     cv::Mat kernel = (cv::Mat_<int>(3, 3) &lt;&lt; 
         -1,  0,  1, 
         -1,  0,  1, 
         -1,  0,  1);

     cv::filter2D(img, o_img, -1, kernel, cv::Point2i(-1, -1), 0.0);
     QImage i_out(o_img.data, o_img.cols, o_img.rows, QImage::Format_BGR888);
     // 2. QPixmap
     QPixmap p_out = QPixmap::fromImage(i_out); 
     // 3. 显示到标签框
     ui-&gt;lbl_video-&gt;setPixmap(p_out);
     // 4. 根据标签框大小显示图像内容
     ui-&gt;lbl_video-&gt;setScaledContents(true);
```

# OpenCV人脸识别(可选)

1. 人脸侦测
2. 标识人脸

- 了解下：
    - 二位码识别

- 代码作业：
    - 完成一个抓取视频的程序（备份上交）
    - 在上一个程序基础上，增加图像效果；
    - 在第一个基础上，识别人脸，并标记；
- 笔记整理
    - 完成课堂笔记（说明，代码，执行效果的截图）

---- 